In [116]:
%display latex
%display default

## Précision numérique

Par défaut, les nombres flottants et les calculs flottants sont fait avec une précision de 53 bits

In [2]:
a= 3.0
a.parent()

Real Field with 53 bits of precision

In [4]:
a.precision()

53

In [5]:
(1.0/3.0).parent()

Real Field with 53 bits of precision

In [6]:
print(1/3, (1/3).parent())
print( (1/3).n(), (1/3).n().parent())
print(RR(1/3), RR(1/3).parent())


1/3 Rational Field
0.333333333333333 Real Field with 53 bits of precision
0.333333333333333 Real Field with 53 bits of precision


Les nombres flottants sont définis comme instance de l'objet `RealField`. On peut spécifier la précision de cette objet.

In [7]:
R=RealField()
R10=RealField(10)
R200=RealField(200)

R.precision(), R10.precision(), R200.precision()

(53, 10, 200)

In [8]:
a   = R(3.513)
a10 = R10(3.513)
a200= R200(3.513)

a^100, a10^100, a200^100

(3.69670656868857e54,
 3.6e54,
 3.6967065686885807935368708162449489665828918404074416864066e54)

Calcul de $0.3-0.2-0.1$ avec différentes précisions

In [9]:
a,b,c=.3,.2,.1
a10,b10,c10=R10(.3),R10(.2),R10(.1)
a200,b200,c200=R200(.3),R200(.2),R200(.1)
a-b-c,a10-b10-c10,a200-b200-c200

(-2.77555756156289e-17,
 -0.00012,
 7.7787690973264271339300800672251553007378152109014589163764e-62)

**Attention:** 
- si on ne précise pas RealField, la précision est calculée automatiquement.
- si les calculs sont mixtes (opérandes avec différentes précisions), les calculs sont fait dans la plus petite précision

In [12]:
a=0.5555555555554033333333323232323233232111117
a.precision(), a.parent()

(143, Real Field with 143 bits of precision)

In [13]:
a10-b200-c200, a10-b10-c10

(-0.00012, -0.00012)

In [14]:
a= 0.1e-20
b= 0.1
c= 0.3e-10
a+b+c-b,b-b+a+c,a+c+b-b

(3.00000024822111e-11, 3.00000000010000e-11, 3.00000024822111e-11)

**A RETENIR**
> - `RealField(p)` est l'objet définissant les approximations flottantes à précision $p$
> - `numerical_approx(expr, p)` ou `n(expr,p)` ou `expr.n(p)` retourne l'approximation flottante de l'expression `expr` à précision `p`
> - par défaut `RealField(53)` est utilisé (déjà défini dans `RR`)

## Exemple de problème de précision

### calcul de racine carré

In [15]:
a=51
b=a^61
print("l'entier b est codée sur "+str( b.nbits()) + " bits")
r=sqrt(b)
r

l'entier b est codée sur 347 bits


1686961934066707040236155036109474174986501789839001*sqrt(51)

In [16]:
s   = R(sqrt(b))
s200= R200(sqrt(b))
print(floor(s),floor(s200))
floor(s)^2 <= b <(floor(s)+1)^2,\
floor(s200)^2 <= b <=(floor(s200)+1)^2

12047317913813611992216795507023424262776490557440000 12047317913813610588023392961569665056411324497659166


(False, True)

## résolution de système d'équations linéaires 


In [17]:
N=2
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=10,den_bound=10)
b=VS.random_element(num_bound=100,den_bound=100)
V=var(list("x"+ str(i) for i in range(N)))
X=vector(V)
eq=list((A*X)[i]==b[i] for i in range(N))

### La fonction `solve`

elle prend une liste d'équations symboliques et une liste d'inconnues

In [19]:
print("variables:",V)
print("Equations:",eq)
for i in range(N):
   print(eq[i])

variables: (x0, x1)
Equations: [-1/2*x0 + 4/5*x1 == (-74/15), 7/6*x0 - 5/2*x1 == (11/28)]
-1/2*x0 + 4/5*x1 == (-74/15)
7/6*x0 - 5/2*x1 == (11/28)


In [20]:
sol=solve(eq, V)
sol

[[x0 == (5048/133), x1 == (14009/798)]]

Est-il possible de faire la même chose mais de faire le calcul numériquement ?

In [21]:
eqf=list((A.n()*X)[i]==b[i].n() for i in range(N))
print("variables:",V)
print("Equations:",eqf)
#for i in range(N):
#    print(eqf[i])

variables: (x0, x1)
Equations: [-0.500000000000000*x0 + 0.800000000000000*x1 == -4.93333333333333, 1.16666666666667*x0 - 2.50000000000000*x1 == 0.392857142857143]


In [22]:
solf=solve(eqf,V)
solf

[[x0 == (5048/133), x1 == (14009/798)]]

Quoi, le résultat est le même !!!

En fait, `solve` fait appel à une méthode de résolution symbolique. 
Cela implique que les réels sont transformées en nombre rationnel lors de la résolution. 

### l'algèbre linéaire

In [24]:
(eq)

[-1/2*x0 + 4/5*x1 == (-74/15), 7/6*x0 - 5/2*x1 == (11/28)]

In [25]:
(A, (V), b)

(
[-1/2  4/5]                           
[ 7/6 -5/2], (x0, x1), (-74/15, 11/28)
)

In [26]:
y=A.solve_right(b)
(y)

(5048/133, 14009/798)

Pareil mais en numérique

In [27]:
Af=A.numerical_approx()
bf=b.n()
(Af,V,bf)

(
[-0.500000000000000  0.800000000000000]          
[  1.16666666666667  -2.50000000000000], (x0, x1),

(-4.93333333333333, 0.392857142857143)
)

In [28]:
yf=Af.solve_right(bf)
(yf)

(37.9548872180451, 17.5551378446115)

In [29]:
# difference des solutions
y.n()-yf

(-1.42108547152020e-14, -1.06581410364015e-14)

Tout pareil mais en plus grand

In [30]:
N=80
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=100,den_bound=100)
b=VS.random_element(num_bound=100,den_bound=100)


In [31]:
#print(A,b)
%time y=A.solve_right(b)

CPU times: user 11.1 s, sys: 31 ms, total: 11.1 s
Wall time: 11.3 s


In [32]:
y[0]

8923822593372696961763036457216188072976669046347296743981805420753415245371899123758966462586694804627668382369312108682394791499160341336976203193162999415288264077550465287200720289404378611748996612733451248084403561081880301232903269863701111367320416989878241746602929802399130322252955267818537975683047686027094175136783876288991561897238260498793586244178630800424125572347578175358411662358316828893329897566973068073439174978689413141191452563657909259544248632467301844344449006656117540638401904807551773738903105546855988502952796778885035867258896470724281081394058741780460770331464682459531451347809563122268074601300051796901947072588798034182228331083639314541295513815522086545039734113681281913045049646692387081337809466598125749183988092338163813181718990643032548105762562838589257209710453516110000131478293527178494539486138564819309319757622104066927919462803303973236866418595002263485620370099486467673720919053049780898613578367640759947315870679526151603593836855468861

In [33]:
print("num(y[0])=",y[0].numerator(),'\n')
print("den(y[0])=",y[0].denominator())

num(y[0])= 89238225933726969617630364572161880729766690463472967439818054207534152453718991237589664625866948046276683823693121086823947914991603413369762031931629994152882640775504652872007202894043786117489966127334512480844035610818803012329032698637011113673204169898782417466029298023991303222529552678185379756830476860270941751367838762889915618972382604987935862441786308004241255723475781753584116623583168288933298975669730680734391749786894131411914525636579092595442486324673018443444490066561175406384019048075517737389031055468559885029527967788850358672588964707242810813940587417804607703314646824595314513478095631222680746013000517969019470725887980341822283310836393145412955138155220865450397341136812819130450496466923870813378094665981257491839880923381638131817189906430325481057625628385892572097104535161100001314782935271784945394861385648193093197576221040669279194628033039732368664185950022634856203700994864676737209190530497808986135783676407599473158706795261516035938

In [34]:
Af=A.n()
bf=b.n()
%time yf= Af.solve_right(bf)

CPU times: user 219 ms, sys: 0 ns, total: 219 ms
Wall time: 212 ms


In [35]:
print("yf[0]=",yf[0])
#erreur commise par la résolution numérique
y[0]-yf[0], (Af*yf)[0]- bf[0]

yf[0]= 0.0185481021483130


(-9.26758669805849e-13, 9.09494701772928e-13)

In [39]:
N=20
A=matrix(N,N,lambda i,j : 1/(i+j+1))
b=vector([ZZ.random_element() for i in range(N)])
((A), b)

(20 x 20 dense matrix over Rational Field,
 (1, 0, -1, -1, 0, -1, -1, 0, -1, 1, 7, 1, 61, 1, -6, -1, 0, 0, 4, 1))

In [40]:
Af=A.n()
bf=b.n()
%time y = A.solve_right(b)
%time yf= Af.solve_right(bf)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.1 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.73 ms


In [43]:
### le résultat avec les flottants est complètement faux
print("yf[0]=",yf[0])
(Af*yf)[0], bf[0], Af*yf==bf

yf[0]= 3.43013324313057e11


(1.66072320000000e7, 1.00000000000000, False)

In [44]:
### le résultat avec les rationnels est exact
print("y[0]=",y[0])
(A*y)[0],b[0],A*y==b

y[0]= 19132637015033500


(1, 1, True)

**A RETENIR**
- la fonction `solve` est une méthode de résolution symbolique
- les calculs avec les flottants peuvent être plus rapides mais aussi faux (surtout en algèbre linéaire)
- les calculs avec les rationnels entrainent un grossissement des données mais le résultat est toujours correct

## Limite du calcul symbolique

In [45]:
var('a b c')
e1=a*c+b*c
e2=(a+b)*c
e1.show()
e2.show()
(e2 == e1.collect(c)).show()
(e1 == e2.expand()).show()
bool(e1==e2)

a*c + b*c

(a + b)*c

(a + b)*c == (a + b)*c

a*c + b*c == a*c + b*c

True

In [46]:
e=(a-b)^2-a^2+2*a*b-b^2
e.show()
bool(e==0)

(a - b)^2 - a^2 + 2*a*b - b^2

True

In [47]:
var('x')
e= cos(x)^2+sin(x)^2
e, e.simplify_trig()

(cos(x)^2 + sin(x)^2, 1)

In [48]:
e=sqrt(x^2)
bool(x==e), bool(x==e.simplify_full()),bool(x==e.canonicalize_radical())

(False, False, True)

### Explosion combinatoire des symboles

In [51]:
var('x')
A=Matrix(2,2,[[cos(x),sin(x)],[-sin(x),cos(x)]])
show("A=",A,"  det(A)=",A.determinant())

'A=' [ cos(x)  sin(x)]
[-sin(x)  cos(x)] '  det(A)=' cos(x)^2 + sin(x)^2

In [54]:
N=100
d=(A^N).determinant()
print("d=",d)

# A^N,(A^(N)).expand()

d= 16*(16*(256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)^2)*(128*(64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 + (256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*si

In [55]:
%time print(bool(d==1))
%time print(bool(d.simplify_trig()==1))

True
CPU times: user 2.94 s, sys: 32 ms, total: 2.97 s
Wall time: 2.95 s
True
CPU times: user 719 ms, sys: 15 ms, total: 734 ms
Wall time: 523 ms


### Pas toujours de solutions

In [56]:
var('x')
(x^2+1).roots()

[(-I, 1), (I, 1)]

In [59]:
E= (x+1)^5 - (x+2)^4
F= (x+1)^5 + (x+2)^4
EF=E*F
print(f"E={E}\nF={F}\nEF={EF}")
EF.roots(x)

E=(x + 1)^5 - (x + 2)^4
F=(x + 1)^5 + (x + 2)^4
EF=((x + 1)^5 + (x + 2)^4)*((x + 1)^5 - (x + 2)^4)


[(-1/6*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3)*(I*sqrt(3) + 1) - 13/3*(1/2)^(2/3)*(-I*sqrt(3) + 1)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (-1/6*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3)*(-I*sqrt(3) + 1) - 13/3*(1/2)^(2/3)*(I*sqrt(3) + 1)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (1/3*(1/2)^(1/3)*(3*sqrt(23)*sqrt(3) + 97)^(1/3) + 26/3*(1/2)^(2/3)/(3*sqrt(23)*sqrt(3) + 97)^(1/3) - 1/3,
  1),
 (-1/2*I*sqrt(3) - 3/2, 1),
 (1/2*I*sqrt(3) - 3/2, 1)]

In [61]:
E= (x+1)^10 - (x+2)^9
F= (x+1)^10 + (x+2)^9
EF=E*F
print(f"E={E}\nF={F}\nEF={EF}")
EF.roots(x) ## aucunes solution trouvées alors que c'est un polynôme

E=(x + 1)^10 - (x + 2)^9
F=(x + 1)^10 + (x + 2)^9
EF=((x + 1)^10 + (x + 2)^9)*((x + 1)^10 - (x + 2)^9)


RuntimeError: no explicit roots found

Pour résoudre on passe du calcul symbolique au calcul algébrique.

Pour cela on se place dans la structure mathématique des polynomes qui sont des expressions symboliques ayant de bonne propriétés: en particulier une représentation canonique !!!

Un polynôme est une expression de la forme : $a_0+a_1X+\dots+a_nX^n$ avec $n$ une valeur connue et les coefficients $\{a_0,a_1,\dots,a_n\}$ qui sont tous du même type (Entier, Rationnel, Complexe, ...).

$\Rightarrow$ le vecteur $[a_0,a_1,\dots,a_n]$ est une représentation canonique de ce polynôme.

In [62]:
# Ensemble des polynomes à coefficient dans QQ
# on choisit la variable X comme symbole des polynômes
PR.<X>=PolynomialRing(QQ) #ou PR.<X>=QQ[]

print("type de x: ",x.parent())
print("type de X: ",X.parent())
bool(X==x)

type de x:  Symbolic Ring
type de X:  Univariate Polynomial Ring in X over Rational Field


False

In [63]:
print((2*X+4).parent()) 
print((2*x+4).parent()) 
print((2*X+ cos(3)).parent())
print((2*X+0.5).parent()) # attention coercion vers les polynomes à coefficient dans RR
RR

Univariate Polynomial Ring in X over Rational Field
Symbolic Ring
Symbolic Ring
Univariate Polynomial Ring in X over Real Field with 53 bits of precision


Real Field with 53 bits of precision

In [65]:
# on convertit les expressions symboliques vers les polynomes en substituant 
# la variable x par celle des polynômes en X et en construisant le polynôme associé
polyE=PR(E.subs(x=X)) 
polyF=PR(F.subs(x=X))
polyEF=polyE*polyF
show(f"une expression symbolique: E={E}")
show(f"un polynomes: polyE={polyE}")


'une expression symbolique: E=(x + 1)^10 - (x + 2)^9'

'un polynomes: polyE=X^10 + 9*X^9 + 27*X^8 - 24*X^7 - 462*X^6 - 1764*X^5 - 3822*X^4 - 5256*X^3 - 4563*X^2 - 2294*X - 511'

In [66]:
# calcul des racines dans le domaine de définition du polynome (ici Q)
print(f"calcul des racines sur {polyEF.base_ring()}")
polyEF.roots()

calcul des racines sur Rational Field


[]

In [67]:
# calcul des racines dans C
R=polyEF.roots(CC)
R

[(-1.51825014565097, 1),
 (4.06351076609262, 1),
 (-1.51929008466337 - 0.389595992999977*I, 1),
 (-1.51929008466337 + 0.389595992999977*I, 1),
 (-1.51922405950470 - 0.270825166290452*I, 1),
 (-1.51922405950470 + 0.270825166290452*I, 1),
 (-1.51876244356380 - 0.171773267885010*I, 1),
 (-1.51876244356380 + 0.171773267885010*I, 1),
 (-1.51838700372900 - 0.0834870580994881*I, 1),
 (-1.51838700372900 + 0.0834870580994881*I, 1),
 (-1.51739709419704 - 0.543838798817926*I, 1),
 (-1.51739709419704 + 0.543838798817926*I, 1),
 (-1.50785565319868 - 0.765883493953419*I, 1),
 (-1.50785565319868 + 0.765883493953419*I, 1),
 (-1.46510584891600 - 1.13480030613066*I, 1),
 (-1.46510584891600 + 1.13480030613066*I, 1),
 (-1.22672212879498 - 1.87991770546555*I, 1),
 (-1.22672212879498 + 1.87991770546555*I, 1),
 (0.520114006346739 - 3.18892454790237*I, 1),
 (0.520114006346739 + 3.18892454790237*I, 1)]

In [70]:
R[0][0],polyEF(R[0][0])

(-1.51825014565097, -9.48784872889519e-9)

In [71]:
polyEF(ComplexField(80)(R[0][0]))

6.3057198351756937881873e-16

In [72]:
# calcul de la forme canonique d'un produit de polynômes
E1= (x+1)^100 - (x+2)^99
F1= (x+1)^100 + (x+2)^99
polyE1=PR(E.subs(x=X)) 
polyF1=PR(F.subs(x=X))

# via la representation par arbre (exp symbolique)
%time (E1*F1).expand() 
# via la representation par liste des coefficients
%time polyEF1=polyE1*polyF1 

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 10.8 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.4 µs


**A RETENIR**
- le calcul symbolique permet de manipuler n'importe quelles expressions faisant intervenir des symboles et des fonctions 
- pas toujours possible de trouver des solutions avec uniquement des calculs symboliques
- l'utilisation de calcul algèbrique est souvent préférable si on connaît la structure mathématique sous-jacente (Ex. les polynômes, les matrices, ...)
- le calcul algébrique sera la plupart du temps plus rapide
- parfois on retrouve un mélange de calcul algébrique et numérique (résolution dans les complexes si le problème est défini sur les nombres rationnels)

## Les matrices

In [77]:
A=matrix([[1,2],[3,4]])
B=matrix([[1,2,3],[4,5,6]])
print(A,B, sep='\n')
show(A,B)

[1 2]
[3 4]
[1 2 3]
[4 5 6]


[1 2]
[3 4] [1 2 3]
[4 5 6]

In [78]:
# addition de matrice
show(A+A)
# multiplication par un scalaire
show(3*A , A+A+A)
print(3*A , A+A+A, sep='\n')

[2 4]
[6 8]

[ 3  6]
[ 9 12] [ 3  6]
[ 9 12]

[ 3  6]
[ 9 12]
[ 3  6]
[ 9 12]


In [79]:
show("A=",A)
show("B=",B)

A+B

'A=' [1 2]
[3 4]

'B=' [1 2 3]
[4 5 6]

TypeError: unsupported operand parent(s) for +: 'Full MatrixSpace of 2 by 2 dense matrices over Integer Ring' and 'Full MatrixSpace of 2 by 3 dense matrices over Integer Ring'

### matrices et combinaisons linéaires

In [80]:
# une combinaison linéaire de symboles
var('x y z')
e=[x,y,z]
3*e[0]+2*e[1]+5*e[2]

3*x + 2*y + 5*z

In [82]:
u=matrix(1,3,[1,2,3])
v=matrix(3,1,e)
show(u,v)
print(u,v, sep='\n')

[1 2 3] [x]
[y]
[z]

[1 2 3]
[x]
[y]
[z]


In [86]:
# le produit de matrice 1*n par n*1 est une combinaison linéaire
u*v

[x + 2*y + 3*z]

In [89]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une ligne à gauche)
B=matrix(2,3,[
    1,2,3,
    4,5,6
    ])
show(B,v)
print(B,v, sep='\n')
show(B*v)

[1 2 3]
[4 5 6] [x]
[y]
[z]

[1 2 3]
[4 5 6]
[x]
[y]
[z]


[  x + 2*y + 3*z]
[4*x + 5*y + 6*z]

In [94]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une colonne à droite)
var('x,y,z,a,b,c')
V=matrix(3,2,[
    x,a,
    y,b,
    z,c
    ])
show(V)
show(B*V)

[x a]
[y b]
[z c]

[  x + 2*y + 3*z   a + 2*b + 3*c]
[4*x + 5*y + 6*z 4*a + 5*b + 6*c]

In [99]:
# ATTENTION: les tailles des matrices doivent correspondre
show(V)
V*V

[x a]
[y b]
[z c]

TypeError: unsupported operand parent(s) for *: 'Full MatrixSpace of 3 by 2 dense matrices over Symbolic Ring' and 'Full MatrixSpace of 3 by 2 dense matrices over Symbolic Ring'

In [97]:
# ATTENTION: le produit matriciel n'est pas commutatif
bool(V*B==B*V)

False

In [102]:
show(V*B,B*V)
print(V*B,B*V, sep='\n\n')

[  4*a + x 5*a + 2*x 6*a + 3*x]
[  4*b + y 5*b + 2*y 6*b + 3*y]
[  4*c + z 5*c + 2*z 6*c + 3*z] [  x + 2*y + 3*z   a + 2*b + 3*c]
[4*x + 5*y + 6*z 4*a + 5*b + 6*c]

[  4*a + x 5*a + 2*x 6*a + 3*x]
[  4*b + y 5*b + 2*y 6*b + 3*y]
[  4*c + z 5*c + 2*z 6*c + 3*z]

[  x + 2*y + 3*z   a + 2*b + 3*c]
[4*x + 5*y + 6*z 4*a + 5*b + 6*c]


In [103]:
# On peut transposer une matrice câd intervertir A[i,j] avec A[j,i]
show(V)
show(V.transpose()) # ou V.T pour la transposition

[x a]
[y b]
[z c]

[x y z]
[a b c]

In [104]:
# le produit d'une matrice par sa transposée est toujours bien défini
show(V.transpose()*V)
show(V*V.transpose())

[x^2 + y^2 + z^2 a*x + b*y + c*z]
[a*x + b*y + c*z a^2 + b^2 + c^2]

[a^2 + x^2 a*b + x*y a*c + x*z]
[a*b + x*y b^2 + y^2 b*c + y*z]
[a*c + x*z b*c + y*z c^2 + z^2]

### Les espaces de matrices

In [105]:
# le premier paramètres est le domaine des coefficients (optionnel)
# si il est présent il impose le type des coefficients
A=matrix([[1,2],[3,4]])
B=matrix(RR,[[1,2/3],[3,4]])
C=matrix([[1,2/3],[3,4]])
A.parent(),B.parent(),C.parent()

(Full MatrixSpace of 2 by 2 dense matrices over Integer Ring,
 Full MatrixSpace of 2 by 2 dense matrices over Real Field with 53 bits of precision,
 Full MatrixSpace of 2 by 2 dense matrices over Rational Field)

In [106]:
matrix(ZZ,[[1,2/3],[3,4]]) # 2/3 not an Integer

TypeError: no conversion of this rational to integer

In [107]:
# le second/troisièmes paramètres sont les dimensions (optionnel)
# si ils sont présents ils imposent le nombre de coefficients (si aucun que des zéros)
A=matrix(ZZ,2,2)
B=matrix(ZZ,2,2,[[4,5],[6,7]])
C=matrix(ZZ,2,2,[4,5,6,7])
show(A)
show(B)
show(C)

[0 0]
[0 0]

[4 5]
[6 7]

[4 5]
[6 7]

In [130]:
# Comme les matrices vivent dans un espace de matrice on peut définir cet objet
MS23=MatrixSpace(ZZ,2,3)
MS22=MatrixSpace(QQ,2,2)
show(MS22,'et',MS23)
MS22,MS23

Full MatrixSpace of 2 by 2 dense matrices over Rational Field 'et' Full MatrixSpace of 2 by 3 dense matrices over Integer Ring

(Full MatrixSpace of 2 by 2 dense matrices over Rational Field,
 Full MatrixSpace of 2 by 3 dense matrices over Integer Ring)

In [128]:
MS23.dimension() , MS23.dims()

(6, (2, 3))

In [123]:
A=MS22.matrix([1,2,3,4])
A.parent()

Full MatrixSpace of 2 by 2 dense matrices over Rational Field

In [132]:
MS22.random_element()
# show(_)

[  -2    1]
[   0 -1/2]

In [134]:
MS10=MatrixSpace(ZZ,10,10)
MS10.random_element(density=1,x=0,y=100)
# show(_)

[60 24 57 51 19 33 98 33 92 37]
[66  6 64 73 28 25 96 77 33 72]
[78 46 72 96  7  5 59 93 79 33]
[58 68 21 78 92 56  1 43 57  8]
[95 35 10 30 60 25 18 65 51 40]
[40 99 61 71 73 45 90 43 20 48]
[77  5 38 65 50 21 96 56 87 64]
[91 18 38 55 17 48 60 90  9 18]
[32 16 79 86 37 77 22 14 20 73]
[38 13  3  4 29 11  2 20 15 61]

In [135]:
MS10=MS10.change_ring(QQ)
MS10.random_element(density=0.5,num_bound=100,den_bound=100)
# show(_)

[ -9/49      0   -2/3   57/4      0 -16/29      0      0      0      0]
[     0  18/71      0 -31/19      0      0  73/28      0  78/47      0]
[     0      0  38/29      0      0      0  37/19      0 21/100      0]
[     0  11/42      0 -64/33      0 -27/13  -7/50      0    6/5      0]
[     0      0 -13/11      0      0 -74/43      0      0 -43/66      0]
[     0      0      0 -53/67   74/9      0      0 -80/81      0      0]
[ 68/11      0      0  43/56  75/53      0   -1/2      0      0      0]
[     0   5/63      0      0      0      0      0  55/18   -7/5  -19/8]
[     0      0  74/33 -97/83    1/3      0  73/76      0      0      0]
[     0      0      0     -5      0  33/16      0  -31/6     61  83/72]

### un peu de calcul algébrique matriciel

In [136]:
N=10
q=17
MS=MatrixSpace(GF(q),N,N)
A=MS.random_element()
show("A=",A)
show(f"det(A)={A.det()}")

'A=' [12  0  4 15  9  8  2 13 15 10]
[ 5  1  4  5  9  6 12 13  2 12]
[13  4 10  2  2  3  6 11  2  1]
[13  8 10  2  3 13  1 16  5 15]
[ 3  6  5 11  6 11  7 13 11 10]
[ 1  0 10  6 15  9  9 11  8  9]
[16  8 14  8 12 13  0  5  0  9]
[ 6  7  2 12  3  0  2  2 14 13]
[10  4 15  8  4  2  7 15 16  2]
[ 0  5 16 14 15 14  9  7 16  1]

'det(A)=15'

In [137]:
B=A.change_ring(ZZ)
show("B=",B)
show(f"det(B)={B.det()}")

'B=' [12  0  4 15  9  8  2 13 15 10]
[ 5  1  4  5  9  6 12 13  2 12]
[13  4 10  2  2  3  6 11  2  1]
[13  8 10  2  3 13  1 16  5 15]
[ 3  6  5 11  6 11  7 13 11 10]
[ 1  0 10  6 15  9  9 11  8  9]
[16  8 14  8 12 13  0  5  0  9]
[ 6  7  2 12  3  0  2  2 14 13]
[10  4 15  8  4  2  7 15 16  2]
[ 0  5 16 14 15 14  9  7 16  1]

'det(B)=8251496565'

In [138]:
B.det()%q == A.det()

True

In [139]:
show(A^-1)

[10  0  7 13  7  5  7  1 14  6]
[ 2  3 13 14  0 14  9 11 10 14]
[15  3  0  8 16  8  7 12  9  7]
[ 3  9  4  9  8 14  3 13 13  9]
[ 3 12  8  0 11 14  4  6  1 10]
[10  7 13  1  1  3 14 16  0 12]
[ 2  6  4  8  9  4 13 14 13 13]
[ 7 10 15  6  3 15 10 14  9 15]
[ 2  0  0  4  5  3 16  8  1  1]
[ 2  4 14 11  2 15  3 10  8  4]

In [140]:
show(B^-1)

[  164630192/2750498855  -111166026/2750498855    107246751/550099771     22691154/550099771  -275297744/2750498855    62263732/2750498855  -212790767/2750498855   168054338/2750498855    -79104828/550099771   163682728/2750498855]
[ -256484821/2750498855 -1117705292/2750498855    213432123/550099771    -74213554/550099771   919842647/2750498855  1029718154/2750498855   -45737384/2750498855   308066371/2750498855   -105774177/550099771  -525206804/2750498855]
[  -57955862/1650299313   489335336/1650299313  -667237556/1650299313     31005220/550099771  -253898692/1650299313  -465522128/1650299313   199512344/1650299313  -166340833/1650299313   474674401/1650299313   125694247/1650299313]
[  -47867399/8251496565  1956691397/8251496565  -598020259/1650299313    -40155470/550099771   373286273/8251496565 -2145859589/8251496565  1415957279/8251496565  -946220026/8251496565   430811795/1650299313  -147540281/8251496565]
[ -134203718/8251496565 -3135221131/8251496565   599119481/1650299313    -72436643/550099771  1787959661/8251496565  3556392322/8251496565  -230765437/8251496565   716495588/8251496565  -341161369/1650299313 -1445729507/8251496565]
[  459549131/8251496565   809554402/8251496565     -376664/1650299313     68668016/550099771 -1191831497/8251496565 -1161104239/8251496565  -628197101/8251496565  -127441841/8251496565  -147514763/1650299313  1395354179/8251496565]
[  104535988/8251496565  1223884031/8251496565   118394642/1650299313     36381045/550099771 -1302085591/8251496565  -998938997/8251496565  -783440113/8251496565   357988352/8251496565  -137430001/1650299313  1273042747/8251496565]
[ -248818678/8251496565 -1235212421/8251496565    48363124/1650299313    -56627963/550099771  1981908121/8251496565  1319399567/8251496565   472537063/8251496565  -430405637/8251496565    96349057/1650299313 -1504516282/8251496565]
[  116281948/2750498855  -378629559/2750498855    122037016/550099771     36836148/550099771  -206144151/2750498855   330204088/2750498855  -404473763/2750498855   268252747/2750498855    -93573983/550099771   186161752/2750498855]
[  -47373853/8251496565  1725377779/8251496565  -396786656/1650299313     41946219/550099771 -1105173404/8251496565 -1347535528/8251496565   315854503/8251496565  -115084172/8251496565   197062120/1650299313   429072053/8251496565]

In [142]:
(B^(-1)%q), A^-1
# show(_)

(
[10  0  7 13  7  5  7  1 14  6]  [10  0  7 13  7  5  7  1 14  6]
[ 2  3 13 14  0 14  9 11 10 14]  [ 2  3 13 14  0 14  9 11 10 14]
[15  3  0  8 16  8  7 12  9  7]  [15  3  0  8 16  8  7 12  9  7]
[ 3  9  4  9  8 14  3 13 13  9]  [ 3  9  4  9  8 14  3 13 13  9]
[ 3 12  8  0 11 14  4  6  1 10]  [ 3 12  8  0 11 14  4  6  1 10]
[10  7 13  1  1  3 14 16  0 12]  [10  7 13  1  1  3 14 16  0 12]
[ 2  6  4  8  9  4 13 14 13 13]  [ 2  6  4  8  9  4 13 14 13 13]
[ 7 10 15  6  3 15 10 14  9 15]  [ 7 10 15  6  3 15 10 14  9 15]
[ 2  0  0  4  5  3 16  8  1  1]  [ 2  0  0  4  5  3 16  8  1  1]
[ 2  4 14 11  2 15  3 10  8  4], [ 2  4 14 11  2 15  3 10  8  4]
)

In [143]:
# Expliquer le résultat obtenu
(B^(-1)%q) ==  A^-1

False

In [ ]:
# GF(N) is the the Integer Ring modulo N, so every elements of matrix A is basically the same as every elements in matrix B (in the Integer Ring) modulo N (work as well for the determinant and every result computed from A)

In [144]:
(B^(-1)%q).parent(),(A^-1).parent()

(Full MatrixSpace of 10 by 10 dense matrices over Rational Field,
 Full MatrixSpace of 10 by 10 dense matrices over Finite Field of size 17)

In [145]:
(B^(-1)%q) ==  (A^-1).change_ring(B.base_ring())

True

In [146]:
(B^(-1)).change_ring(A.base_ring()) ==  (A^-1)

True